In [1]:
import os 
import yaml
from tensorflow.keras import preprocessing,utils
import numpy as np
import tensorflow.keras as k
import string

2023-05-25 01:44:00.766331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 01:44:01.111147: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 01:44:01.166541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-25 01:44:01.166568: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
path='/home/mohamedtez/Desktop/PCD/tutor1/data'
files=os.listdir(path)
questions=[]
answers=[]
for file in files:
    data=open(path+'/'+file,'rb')
    docs=yaml.safe_load(data)
    conversations=docs['conversations']
    for dialogue in conversations:
        if (len(dialogue)>2):
            questions.append(dialogue[0])
            ans=dialogue[1:]
            responses=""
            for res in ans:
                responses+=' '+res
            answers.append(responses)
        elif len(dialogue)==2:
            questions.append(dialogue[0])
            answers.append(dialogue[1])
## questions list of all the question
## responses list of all the answers

In [3]:
encoder_questions=questions
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(encoder_questions)
tkz_encoder_questions=tokenizer.texts_to_sequences(encoder_questions)
questions
longest_question=0
for que in tkz_encoder_questions:
    if len(que)>longest_question:
        longest_question=len(que)
padded_tkz_encoder_questions=preprocessing.sequence.pad_sequences(tkz_encoder_questions,maxlen=longest_question,padding='post')
encoder_questions=np.array(padded_tkz_encoder_questions)
encoder_questions.shape
question_word_dict=tokenizer.word_index
question_word_dict
question_tokens=len(question_word_dict)+1

In [4]:
decoder_answers=[]
for ans in answers:
    decoder_answers.append('<START>'+ans+'<END>')
    
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(decoder_answers)
tkz_decoder_answers=tokenizer.texts_to_sequences(decoder_answers)

longest_answer=0
for ans in tkz_decoder_answers:
    if len(ans)>longest_answer:
        longest_answer=len(ans)
padded_tkz_decoder_answers=preprocessing.sequence.pad_sequences(tkz_decoder_answers,maxlen=longest_answer,padding='post')
padded_decoder=np.array(padded_tkz_decoder_answers)
padded_decoder.shape
answer_word_dict=tokenizer.word_index
answer_word_dict
answers_tokens=len(answer_word_dict)+1

In [5]:
decoder_target_data=[]
for token in tkz_decoder_answers:
    decoder_target_data.append(token[1:])
temp=preprocessing.sequence.pad_sequences(decoder_target_data,maxlen=longest_answer,padding='post')

onehot_padded_answers=utils.to_categorical(temp,answers_tokens)
decoder_target_data=np.array(onehot_padded_answers)
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [6]:
encoder_inputs=k.layers.Input(shape=(None,))
encoder_embedding=k.layers.Embedding(question_tokens,200,mask_zero=True)(encoder_inputs)
encoder_outputs,state_h,state_c=k.layers.LSTM(200,return_state=True)(encoder_embedding)
encoder_states=[state_h,state_c]

decoder_inputs=k.layers.Input(shape=(None,))
decoder_embedding=k.layers.Embedding(answers_tokens,200,mask_zero=True)(decoder_inputs)
decoder_lstm=k.layers.LSTM(200,return_state=True,return_sequences=True)
decoder_output, _,_ =decoder_lstm(decoder_embedding,initial_state=encoder_states)
decoder_dense=k.layers.Dense(answers_tokens,activation=k.activations.softmax)
output=decoder_dense(decoder_output)



model=k.models.Model([encoder_inputs,decoder_inputs],output)
model.compile(optimizer=k.optimizers.RMSprop(),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

2023-05-25 01:44:09.907870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-25 01:44:09.907912: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-25 01:44:09.907942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mohamedtez-Inspiron-15-3511): /proc/driver/nvidia/version does not exist
2023-05-25 01:44:09.908735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 200)    138000      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 200)    459000      ['input_2[0][0]']                
                                                                                              

In [37]:
##this is not compiled 
model.fit([encoder_questions,padded_decoder],decoder_target_data,batch_size=300,epochs=300)

Epoch 1/300
3/3 [==============================] - 6s 2s/step - loss: 0.8199 - accuracy: 0.8468
Epoch 2/300
3/3 [==============================] - 6s 2s/step - loss: 0.8265 - accuracy: 0.8451
Epoch 3/300
3/3 [==============================] - 5s 2s/step - loss: 0.8221 - accuracy: 0.8470
Epoch 4/300
3/3 [==============================] - 5s 2s/step - loss: 0.8212 - accuracy: 0.8459
Epoch 5/300
3/3 [==============================] - 5s 2s/step - loss: 0.8177 - accuracy: 0.8484
Epoch 6/300
3/3 [==============================] - 5s 2s/step - loss: 0.8182 - accuracy: 0.8480
Epoch 7/300
3/3 [==============================] - 6s 2s/step - loss: 0.8167 - accuracy: 0.8495
Epoch 8/300
3/3 [==============================] - 6s 2s/step - loss: 0.8320 - accuracy: 0.8415
Epoch 9/300
3/3 [==============================] - 5s 2s/step - loss: 0.8197 - accuracy: 0.8483
Epoch 10/300
3/3 [==============================] - 5s 2s/step - loss: 0.8058 - accuracy: 0.8493
Epoch 11/300
3/3 [=====================

3/3 [==============================] - 6s 2s/step - loss: 0.6954 - accuracy: 0.8786
Epoch 86/300
3/3 [==============================] - 5s 2s/step - loss: 0.6970 - accuracy: 0.8773
Epoch 87/300
3/3 [==============================] - 5s 2s/step - loss: 0.7001 - accuracy: 0.8785
Epoch 88/300
3/3 [==============================] - 6s 2s/step - loss: 0.6951 - accuracy: 0.8783
Epoch 89/300
3/3 [==============================] - 6s 2s/step - loss: 0.6899 - accuracy: 0.8814
Epoch 90/300
3/3 [==============================] - 6s 2s/step - loss: 0.6887 - accuracy: 0.8798
Epoch 91/300
3/3 [==============================] - 5s 2s/step - loss: 0.6878 - accuracy: 0.8801
Epoch 92/300
3/3 [==============================] - 5s 2s/step - loss: 0.6863 - accuracy: 0.8812
Epoch 93/300
3/3 [==============================] - 5s 2s/step - loss: 0.7027 - accuracy: 0.8768
Epoch 94/300
3/3 [==============================] - 5s 2s/step - loss: 0.6862 - accuracy: 0.8815
Epoch 95/300
3/3 [=========================

3/3 [==============================] - 5s 2s/step - loss: 0.5830 - accuracy: 0.9074
Epoch 169/300
3/3 [==============================] - 5s 2s/step - loss: 0.5905 - accuracy: 0.9049
Epoch 170/300
3/3 [==============================] - 5s 2s/step - loss: 0.5938 - accuracy: 0.9044
Epoch 171/300
3/3 [==============================] - 6s 2s/step - loss: 0.5981 - accuracy: 0.9020
Epoch 172/300
3/3 [==============================] - 6s 2s/step - loss: 0.5866 - accuracy: 0.9059
Epoch 173/300
3/3 [==============================] - 6s 2s/step - loss: 0.5796 - accuracy: 0.9063
Epoch 174/300
3/3 [==============================] - 6s 2s/step - loss: 0.5794 - accuracy: 0.9076
Epoch 175/300
3/3 [==============================] - 6s 2s/step - loss: 0.5849 - accuracy: 0.9051
Epoch 176/300
3/3 [==============================] - 6s 2s/step - loss: 0.5876 - accuracy: 0.9033
Epoch 177/300
3/3 [==============================] - 5s 2s/step - loss: 0.5857 - accuracy: 0.9053
Epoch 178/300
3/3 [===============

3/3 [==============================] - 6s 2s/step - loss: 0.4993 - accuracy: 0.9271
Epoch 252/300
3/3 [==============================] - 6s 2s/step - loss: 0.5075 - accuracy: 0.9250
Epoch 253/300
3/3 [==============================] - 5s 2s/step - loss: 0.5040 - accuracy: 0.9248
Epoch 254/300
3/3 [==============================] - 5s 2s/step - loss: 0.4913 - accuracy: 0.9289
Epoch 255/300
3/3 [==============================] - 6s 2s/step - loss: 0.4911 - accuracy: 0.9288
Epoch 256/300
3/3 [==============================] - 5s 2s/step - loss: 0.4903 - accuracy: 0.9296
Epoch 257/300
3/3 [==============================] - 5s 2s/step - loss: 0.4922 - accuracy: 0.9289
Epoch 258/300
3/3 [==============================] - 5s 2s/step - loss: 0.4976 - accuracy: 0.9275
Epoch 259/300
3/3 [==============================] - 5s 2s/step - loss: 0.4854 - accuracy: 0.9300
Epoch 260/300
3/3 [==============================] - 6s 2s/step - loss: 0.4874 - accuracy: 0.9306
Epoch 261/300
3/3 [===============

In [38]:
def create_encoder_decoder_models():
    ##model -encoder-
    encoder_model=k.models.Model(encoder_inputs,encoder_states)
    ##decoder
    decoder_input_state_h=k.layers.Input(shape=(200,))
    decoder_input_state_c=k.layers.Input(shape=(200,))
    decoder_input_states=[decoder_input_state_h,decoder_input_state_c]

    decoder_output,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=decoder_input_states)

    decoder_output_states=[state_h,state_c]
    decoder_output =decoder_dense(decoder_output)
    decoder_model = k.models.Model([decoder_inputs]+decoder_input_states,[decoder_output]+decoder_output_states)
    return encoder_model,decoder_model

In [39]:
enc_model,dec_model=create_encoder_decoder_models()
enc_model

In [40]:
##this is not compiled  
enc_model.save('/home/mohamedtez/Desktop/PCD/tutor1/encoder_model.h5')
dec_model.save('/home/mohamedtez/Desktop/PCD/tutor1/decoder_model.h5')

In [41]:
test =True
def question_to_tokens(question):
    
    question=question.translate(str.maketrans('','',string.punctuation))
    words=question.lower().split()
    tokens=[]
    for word in words:
        try:
            tokens.append(question_word_dict[word])
        except:
            print("the question is not recognized!")
            test=False
            break
    return preprocessing.sequence.pad_sequences([tokens],maxlen=longest_question,padding='post')

question="hello"
##question_to_tokens(input("please enter your question"))

In [42]:
enc_model=k.models.load_model('/home/mohamedtez/Desktop/PCD/tutor1/encoder_model.h5',compile=False)
dec_model=k.models.load_model('/home/mohamedtez/Desktop/PCD/tutor1/decoder_model.h5',compile=False)

In [47]:
states_values=enc_model.predict(question_to_tokens("cancer type"))##in this line we can test any string
token=np.zeros((1,1))
token[0,0]= answer_word_dict['start']

if test==True:
    stop_condition= False
else:
    stop_condition= True
chatbot_answer=""
while not stop_condition:
    dec_output,h,c=dec_model.predict([token]+states_values)
    index=np.argmax(dec_output[0,0,:])
    word=list(answer_word_dict.keys())[index-1]
    chatbot_answer+=' '+word
    if word == 'end' or len(chatbot_answer.split())>longest_answer:
        stop_condition=True

    token[0,0]=index
    states_values=[h,c]

print(chatbot_answer[:-3])

1/1 [==============================] - 0s 20ms/step
 cancer pain refers to the period of time after cancer treatment has ended during this time survivors may face ongoing physical emotional and financial challenges related to their cancer diagnosis and treatment 
